<a href="https://colab.research.google.com/github/anjali-1002/FFML_Projects_and_Labs/blob/main/Copy_of_FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [3]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [4]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [5]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [6]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [7]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [8]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [9]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [10]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [11]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [12]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [13]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [14]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [15]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Answers:

1.The accuracy of the validation set in a machine learning model can be affected by the size of the validation set, but the effect is somewhat nuanced.

Increasing the Percentage of the Validation Set
Variance of Validation Accuracy: If you increase the size of the validation set, the variance in the validation accuracy generally decreases. This is because a larger validation set provides a more comprehensive sample of the data distribution, leading to more stable and reliable estimates of the model's performance.

Model Tuning and Training: With more data in the validation set, you might have less data available for training. This can potentially lead to less robust training if the training set becomes too small, which might negatively impact the model's performance. However, this is more of a concern when the training set is reduced to the point where it significantly impacts the model's learning process.

Overfitting on Validation Set: Increasing the validation set size reduces the chance of overfitting to the validation set itself, but it's crucial to ensure that the validation set is representative of the data the model will encounter in real-world scenarios.

Reducing the Percentage of the Validation Set
Variance of Validation Accuracy: If you reduce the size of the validation set, the variance in the validation accuracy can increase. A smaller validation set may not represent the full data distribution as well, leading to higher variability in the validation accuracy. This can make it harder to get a reliable estimate of the model's performance.

Training Data Availability: A smaller validation set means that more data is available for training. This can be beneficial as the model can learn from more examples, potentially improving its performance if the training set is now sufficiently large.

Risk of Overfitting: With a smaller validation set, there is a risk of overfitting to the validation set itself, particularly if model tuning decisions are influenced by the limited validation data. This can lead to a situation where the model performs well on the validation set but poorly on unseen data.

Key Takeaways
Larger Validation Set: Typically leads to a more stable and accurate estimate of the model's performance but might reduce the size of the training set, potentially affecting training effectiveness.

Smaller Validation Set: Can lead to higher variance in performance estimates and may not fully capture the model's true performance, but provides more data for training the model.

In practice, finding the right balance between the training set and validation set size is crucial for developing a robust model. The choice often depends on the total amount of data available and the specific requirements of the task at hand.




2.The size of the training and validation sets can significantly influence how well the validation set predicts accuracy on the test set. Here’s a breakdown of the effects:

 Larger Training Set

1. Better Model Performance: A larger training set generally leads to a better-trained model, as it provides more examples for the model to learn from. This can improve the generalization of the model, which can lead to more accurate predictions on the test set.

2. Validation Set Accuracy: When the training set is larger, the validation set, which is used to estimate model performance, may be more indicative of the model's performance on the test set. This is because a well-trained model will have learned to generalize better from the training data, making the validation accuracy a more reliable estimate of test accuracy.

 Larger Validation Set

1. More Reliable Estimate: Increasing the size of the validation set provides a more reliable estimate of the model's performance. A larger validation set better represents the data distribution, reducing variability in performance estimates and providing a clearer picture of how the model will likely perform on the test set.

2. Impact on Training: If increasing the validation set reduces the size of the training set too much, the model may not be as well-trained, which could negatively impact the accuracy of predictions on the test set. In such cases, the validation set may not perfectly reflect the test set performance due to underfitting.

Smaller Training Set

1. Potential Underfitting: A smaller training set might lead to underfitting, where the model does not learn effectively from the data. This can result in a model that does not generalize well, making the validation set less predictive of test set performance.

2. Validation Accuracy Variability: With less data to train on, the model may perform inconsistently, causing the validation set accuracy to be a less stable predictor of the test set accuracy.

 Smaller Validation Set

1. Higher Variability: A smaller validation set can lead to higher variability in performance estimates. This means that the accuracy observed on the validation set might not be as reliable or indicative of test set performance. Small validation sets may not capture the full range of the data distribution, making predictions on the test set less predictable.

2. Overfitting Risk: With a smaller validation set, there's a greater risk that the model might be overfitted to the validation set itself, particularly if model tuning is heavily influenced by the validation performance. This can result in poor generalization to the test set.

 Summary

- Larger Training Set: Generally improves model performance and can lead to a validation set that more accurately predicts test set accuracy.
- Larger Validation Set: Provides a more stable and reliable estimate of model performance, though it must be balanced with adequate training data to avoid underfitting.
- Smaller Training Set: May lead to underfitting and reduced accuracy on the test set, making validation accuracy less predictive.
- Smaller Validation Set: Can lead to higher variability and less reliability in estimating test set performance.

Ultimately, balancing the sizes of the training and validation sets is crucial. The goal is to have enough data for effective training while ensuring the validation set provides a reliable estimate of performance, which in turn helps in accurately predicting test set accuracy.


3.Choosing the right percentage for the validation set involves balancing several factors, including the size of the dataset, the need for a reliable performance estimate, and the amount of data available for training. Here's a general guideline based on common practices:

 Common Practice Recommendations

1. Standard Split:
   - Validation Set: Typically 10% to 20% of the total dataset.
   - Training Set: The remainder (80% to 90%).

   This split is often used because it provides a good balance between having enough data to train the model and having a sufficiently large validation set to estimate performance reliably.

2. Cross-Validation:
   - Instead of using a fixed validation set percentage, you might use k-fold cross-validation. In k-fold cross-validation, the dataset is divided into \( k \) subsets. Each subset is used as a validation set while the remaining \( k-1 \) subsets are used for training. This approach provides a more reliable estimate of model performance by averaging results over multiple splits. Common choices for \( k \) are 5 or 10, which means each fold uses about 20% or 10% of the data for validation, respectively.

3. Alternative Splits:
   - For Large Datasets: If you have a very large dataset, reserving a smaller percentage (e.g., 5% to 10%) for validation might be sufficient. This is because even a small percentage of a large dataset still provides a substantial number of samples.
   - For Smaller Datasets: If your dataset is smaller, you might need to reserve a larger percentage (e.g., 20% to 30%) for validation to ensure that the validation set provides a reliable estimate of performance.

 Factors to Consider

1. Dataset Size:
   - Large Datasets: You can afford to reserve a smaller percentage for validation (e.g., 10%) while still having enough data to train and validate effectively.
   - Small Datasets: You might need a larger percentage for validation (e.g., 20% to 30%) to ensure that the validation set is representative. However, this means the training set will be smaller, which might impact model performance.

2. Model Complexity and Training Requirements:
   - Complex Models: For complex models that require substantial data to train effectively, ensure the training set remains sufficiently large by using a smaller validation set percentage or by employing cross-validation.
   - Simple Models: If the model is relatively simple, you might get away with a larger validation set percentage without impacting training too much.

3. Performance Estimation:
   - To get a reliable estimate of performance, ensure that the validation set is large enough to provide stable and consistent performance metrics.

Summary

- General Guideline: Use 10% to 20% of your data for the validation set if the dataset is reasonably large.
- Cross-Validation: Consider k-fold cross-validation to get a more robust performance estimate.
- Adjust Based on Dataset Size: Increase the validation set percentage if your dataset is small, and be flexible with the split based on the specific needs of your model and data.

Balancing these factors helps ensure that the validation set provides a good estimate of model performance while still leaving enough data for effective training.

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

In [1]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load the data
from sklearn import datasets
dataset = datasets.fetch_california_housing()
X = dataset.data
y = dataset.target.astype(int)  # Convert target to integer for classification

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 1-NN Classifier
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train, y_train)
y_pred_1nn = knn1.predict(X_test)
accuracy_1nn = accuracy_score(y_test, y_pred_1nn)
print(f"1-NN Accuracy: {accuracy_1nn * 100:.2f}%")

# 3-NN Classifier
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(X_train, y_train)
y_pred_3nn = knn3.predict(X_test)
accuracy_3nn = accuracy_score(y_test, y_pred_3nn)
print(f"3-NN Accuracy: {accuracy_3nn * 100:.2f}%")


1-NN Accuracy: 34.82%
3-NN Accuracy: 35.00%


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [16]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [17]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Answers:

1.Yes, averaging the validation accuracy across multiple splits generally provides more consistent and reliable results. This approach is often used in cross-validation techniques, like k-fold cross-validation, to assess a model's performance more accurately.

2.Yes, averaging validation accuracy across multiple splits tends to give a more accurate estimate of test accuracy. This is a core reason why techniques like k-fold cross-validation are widely used in machine learning mode evaluation.

3.he number of iterations (or splits) used in cross-validation can significantly impact the accuracy and reliability of the estimate of the test accuracy. Here’s how increasing the number of iterations affects the estimate:

 1. Reduced Variance and Bias
   - Higher Iterations (More Splits): Increasing the number of iterations typically reduces the variance in the performance estimate. More splits mean more diverse subsets of the data are used for validation, leading to a more stable and representative estimate of the model's performance. It also reduces the bias that might result from relying on a few splits that could potentially be unrepresentative.
   - Lower Iterations (Fewer Splits): With fewer iterations, there's a higher chance that the model's performance estimate is influenced by the particular choice of training and validation sets. This can lead to higher variance in the estimate, as the model might perform well on some splits but poorly on others, depending on how the data is divided.

2. Better Generalization Estimate
   - Higher Iterations:More iterations allow the model to see more variations in the data. As a result, the performance estimate is likely to generalize better to unseen data because the model has been evaluated in more varied scenarios.
   - Lower Iterations: If the number of iterations is too low, the estimate might not capture the full diversity of the dataset, which could lead to an over- or underestimation of the model's true performance.

 3. Computational Cost
   - Higher Iterations: While increasing the number of iterations can provide a more accurate and reliable estimate, it also increases the computational cost. More iterations mean that the model needs to be trained and validated more times, which can be time-consuming, especially with large datasets and complex models.
   - Balancing Act:There's a trade-off between computational efficiency and the accuracy of the estimate. Typically, a commonly used number of iterations, such as 5 or 10 in k-fold cross-validation, strikes a good balance between providing a reliable estimate and managing computational resources.

 4. Diminishing Returns
   - After a certain point, increasing the number of iterations yields diminishing returns in terms of the improvement in the estimate's accuracy. For example, the difference in accuracy between using 10-fold and 20-fold cross-validation is often not substantial enough to justify the extra computational cost. Thus, while more iterations generally improve the estimate's reliability, the gains may level off beyond a certain point.

 5. Leave-One-Out Cross-Validation (LOOCV)
   - As an extreme case, leave-one-out cross-validation (where each data point is used once as a validation set) provides the maximum number of iterations. LOOCV reduces bias to a minimum, but it can have high variance because each training set is very similar to the others, and it is computationally expensive.

Conclusion
Yes, using more iterations generally leads to a better and more reliable estimate of test accuracy by reducing variance and bias. However, it is essential to balance the number of iterations with the available computational resources. A commonly adopted approach is to use 5-fold or 10-fold cross-validation, which offers a good balance between reliability and efficiency.

4.Increasing the number of iterations in cross-validation can help to some extent when dealing with a very small training or validation dataset, but it has limitations  and trade-offs.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

In [2]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Function to evaluate and compare 1-NN and 3-NN classifiers with varying split sizes
def compare_knn_classifiers(X, y, splits, iterations):
    results = {
        '1-NN': [],
        '3-NN': []
    }

    for split_size in splits:
        acc_1nn = []
        acc_3nn = []

        for _ in range(iterations):
            # Split the data
            X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=split_size)

            # 1-NN Classifier
            knn1 = KNeighborsClassifier(n_neighbors=1)
            knn1.fit(X_train, y_train)
            y_pred_1nn = knn1.predict(X_val)
            acc_1nn.append(accuracy_score(y_val, y_pred_1nn))

            # 3-NN Classifier
            knn3 = KNeighborsClassifier(n_neighbors=3)
            knn3.fit(X_train, y_train)
            y_pred_3nn = knn3.predict(X_val)
            acc_3nn.append(accuracy_score(y_val, y_pred_3nn))

        # Store the average accuracy for each split size
        results['1-NN'].append(np.mean(acc_1nn))
        results['3-NN'].append(np.mean(acc_3nn))

    return results

# Load the data
dataset = datasets.fetch_california_housing()
X = dataset.data
y = dataset.target.astype(int)

# Define split sizes and iterations
split_sizes = [0.6, 0.7, 0.8]  # 60%, 70%, 80% of the data used for training
iterations = 10  # Number of iterations for averaging

# Perform the comparison
results = compare_knn_classifiers(X, y, splits=split_sizes, iterations=iterations)

# Print the results
for split_size, acc_1nn, acc_3nn in zip(split_sizes, results['1-NN'], results['3-NN']):
    print(f"Split size: {split_size*100:.2f}%")
    print(f"1-NN Average Accuracy: {acc_1nn*100:.2f}%")
    print(f"3-NN Average Accuracy: {acc_3nn*100:.2f}%\n")

Split size: 60.00%
1-NN Average Accuracy: 34.13%
3-NN Average Accuracy: 34.41%

Split size: 70.00%
1-NN Average Accuracy: 34.65%
3-NN Average Accuracy: 34.90%

Split size: 80.00%
1-NN Average Accuracy: 34.69%
3-NN Average Accuracy: 35.28%

